In [26]:
import pandas as pd
eeg = pd.read_csv("C:/Users/Admin/Desktop/cognitive load/Data/EEG.csv")
gsr = pd.read_csv("C:/Users/Admin/Desktop/cognitive load/Data/GSR.csv")
psy = pd.read_csv("C:/Users/Admin/Desktop/cognitive load/Data/PSY.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_16144\2163068185.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  eeg = pd.read_csv("C:/Users/Admin/Desktop/cognitive load/Data/EEG.csv")
C:\Users\Admin\AppData\Local\Temp\ipykernel_16144\2163068185.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  psy = pd.read_csv("C:/Users/Admin/Desktop/cognitive load/Data/PSY.csv")


In [27]:
import numpy as np

def simple_eeg_features(df):
    features = {}

    features['delta_mean'] = df[['Delta_TP9','Delta_AF7','Delta_AF8','Delta_TP10']].mean().mean()
    features['theta_mean'] = df[['Theta_TP9','Theta_AF7','Theta_AF8','Theta_TP10']].mean().mean()
    features['alpha_mean'] = df[['Alpha_TP9','Alpha_AF7','Alpha_AF8','Alpha_TP10']].mean().mean()
    features['beta_mean']  = df[['Beta_TP9','Beta_AF7','Beta_AF8','Beta_TP10']].mean().mean()
    features['gamma_mean'] = df[['Gamma_TP9','Gamma_AF7','Gamma_AF8','Gamma_TP10']].mean().mean()

    return features


In [28]:
def simple_gsr_features(df):
    features = {}
    features['gsr_mean'] = df['GSR Conductance CAL'].mean()
    features['gsr_std'] = df['GSR Conductance CAL'].std()
    return features


In [29]:
print(psy.columns.tolist())
psy.columns = psy.columns.str.strip()
eeg.columns = eeg.columns.str.strip()
gsr.columns = gsr.columns.str.strip()
psy['TimeStamp'] = pd.to_datetime(psy['TimeStamp'].astype(str), errors='coerce')
eeg['TimeStamp'] = pd.to_datetime(eeg['TimeStamp'].astype(str), errors='coerce')
gsr['TimeStamp'] = pd.to_datetime(gsr['TimeStamp'].astype(str), errors='coerce')

psy = psy.dropna(subset=['TimeStamp'])
eeg = eeg.dropna(subset=['TimeStamp'])
gsr = gsr.dropna(subset=['TimeStamp'])
print(type(psy.iloc[0]['TimeStamp']))


['TimeStamp', 'UnixTime', 'QuestionKey', 'QuestionKey.1', 'dataset_id', 'Elements']
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [32]:
psy=psy.head(50)
rows = []

for i in range(0, len(psy)):
    start_time = psy.iloc[i]['TimeStamp']
    end_time = start_time + pd.Timedelta(seconds=20)

    eeg_slice = eeg[(eeg['TimeStamp'] >= start_time) & (eeg['TimeStamp'] <= end_time)]
    gsr_slice = gsr[(gsr['TimeStamp'] >= start_time) & (gsr['TimeStamp'] <= end_time)]

    if len(eeg_slice) == 0 or len(gsr_slice) == 0:
        continue

    eeg_feat = simple_eeg_features(eeg_slice)
    gsr_feat = simple_gsr_features(gsr_slice)

    # Simple labels (student-friendly)
    label = 1 if i % 2 == 0 else 0   # dummy low/high

    rows.append({**eeg_feat, **gsr_feat, 'label': label})

df_features = pd.DataFrame(rows)
df_features.to_csv("C:/Users/Admin/Desktop/cognitive load/Data/features.csv", index=False)

df_features.head()


,delta_mean,theta_mean,alpha_mean,beta_mean,gamma_mean,gsr_mean,gsr_std,label
0,0.672914,0.340504,0.613938,0.310113,-0.092388,16.348758,0.924684,1
1,0.672900,0.340568,0.613935,0.310113,-0.092406,16.348923,0.924700,0
2,0.672896,0.340602,0.613939,0.310101,-0.092429,16.349108,0.924662,1
3,0.672895,0.340670,0.613951,0.310077,-0.092475,16.349477,0.924586,0
4,0.672891,0.340704,0.613960,0.310066,-0.092498,16.349663,0.924546,1
